In [13]:
import re
import os
import numpy as np
import string as str
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from hmmlearn import hmm
from collections import defaultdict

#nltk.download('punkt') 

directory = "../data/"
SWords = stopwords.words('french')
SWords.extend(list(str.punctuation))
filtre_fr = lambda text: [token for token in text if token.lower() not in SWords]

def clean_text(text): #texte string
    ctext = re.sub(r'<.*?>', '', text)
    ctext = ctext.replace('/', '')
    ctext = ctext.replace('.', ' ')
    ctext = ctext.replace(',', '')
    for w in ctext:
        if w.lower() in SWords:
            ctext.replace(w, '')
    return ctext

def create_yearfile(year):
    outputfile = 'ECD_'+ year + '.txt'
    with open(outputfile, 'w', encoding='utf8') as outfile:
        for filename in os.listdir(directory):
            if filename.startswith(year):
                with open(os.path.join(directory, filename), 'r', encoding='utf-8') as infile:
                    text = infile.read()  # Lire le fichier
                    ctext = clean_text(text)
                    outfile.write(ctext)
                    outfile.write('\n')

    return outputfile


In [17]:
year = '3E100'
outputfile = 'ECD_'+ year + '.txt'
txtfile = '/Analyse du corpus/data/3E18_0023.txt'

outputfile = create_yearfile(year)

with open(txtfile, 'r', encoding="utf8") as file:
    data = file.read()
    #pour fichier unique
    ctext = clean_text(data)

print(ctext)

data = data.lower()
fwords = filtre_fr(word_tokenize(data, language='french'))
couples_w = [(fwords[i], fwords[i+1]) for i in range(len(fwords)-1)]
words_unique = list(set(fwords))
words_unique.sort()
fd = nltk.FreqDist(fwords)
frequence = fd.most_common()

# Création d'un dictionnaire pour mapper chaque mot à un entier (requis pour l'algorithme)
w_to_int = {w: i for i, w in enumerate(words_unique)}
int_to_w = {i: w for w, i in w_to_int.items()}

couples_id = [(w_to_int[mot1], w_to_int[mot2]) for mot1, mot2 in couples_w]


N°103
Mathieu 
Tunisson
L’An mil huit cent Sept le vingt cinq du mois de mai à onze 
heures du matin sur un billet en date de ce jour signe Delacour 
pour l’Econome de l’hopital militaire de cette commune Situe faubourg 
Voltaire portant que le nommé Mathieu Tunisson Soldat au 
treizième régiment de chasseur à cheval natif de Kevelaer 
Canton de Gueldre Département de la Roër entré au dit hospice 
le vingt deux du mois de mai courant y est mort ce jourd’hui 
nous Jean Claude Gerard Maire et officier de l’Etat Civil 
de la commune de Belfort Département du haut Rhin avons 
dressé le présent acte de Décès que nous avons signé    
militaire

N°104
Enfant sans vie 
né de Ursule
L’homme et 
de françois 
Louis Mouchot 
L’An mil huit Cent Sept le vingt huit du mois de mai à neuf 
heures du matin pardevant nous Jean Claude Gerard maire et officier 
de l’état Civil de la Commune de Belfort Département du haut-rhin 
ont Comparu françois Louis Mouchot âgé de quarante trois ans 
Chaudronnier domic

### Création de la matrice de transition

In [11]:
n = len(words_unique)
matrice_transition = np.zeros((n, n))

# Remplissage de la matrice de transition avec les comptages
for (indice_wt, indice_wtp1) in couples_id:
    matrice_transition[indice_wtp1, indice_wt] += 1

# Mise à zéro de la diagonale
#np.fill_diagonal(matrice_transition, 0)

# Normalisation par la somme de chaque colonne pour obtenir des probabilités
with np.errstate(divide='ignore', invalid='ignore'):  # Pour gérer la division par zéro
    matrice_probabilites = matrice_transition / matrice_transition.sum(axis=0, keepdims=True)
    matrice_probabilites = np.nan_to_num(matrice_probabilites)  # Remplace les NaN par 0


In [6]:

# Transformez les mots en indices d'états
observations = np.array(couples_id).reshape(-1, 2)

# Création d'un modèle HMM avec un nombre d'états correspondant au nombre de mots uniques
model = hmm.MultinomialHMM(n_components=n_states, n_iter=100)

# Initialisation d'une matrice de probabilité égale (ou basée sur la fréquence de mots si disponible)
start_prob = np.full(n_states, 1.0 / n_states)  # Probabilité de départ uniforme
transition_prob = np.full((n_states, n_states), 1.0 / n_states)  # Probabilités de transition égales

model.startprob_ = start_prob
model.transmat_ = transition_prob

# Ajustement du modèle sur les observations
model.fit(observations)

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Fitting a model with 77840 free scalar parameters with only 1275 data points will result in a degenerate solution.


MultinomialHMM(n_components=279, n_iter=100,
               n_trials=array([247, 129, 145, ...,  11, 151,  99]),
               random_state=RandomState(MT19937) at 0x1675BD7F040)

In [16]:
# Prédiction : prédire le mot suivant après un mot donné
def predire_mot_suivant(mot):
    idx = w_to_int[mot]
    next_word_idx = model.predict([[idx]])[0]
    return int_to_w[next_word_idx]

# Exemple de prédiction du mot suivant
mot_initial = "maire"
mot_suivant = predire_mot_suivant(mot_initial)
print(f"Mot suivant après '{mot_initial}': {mot_suivant}")

Mot suivant après 'maire': a
